In [ ]:
import requests
import pandas as pd
from IPython.display import display
import re

API_KEY = "01946b8515c545443cdcd262a884a88dab1be54962aad37f4f93c3420cc49844"
headers = {"X-API-Key": API_KEY}

def extract_data (url): 

    params = {
        "limit": 1000,
        "page": 1
    }

    all_results = []

    while True:
        response = requests.get(url, headers=headers, params=params)
        if response.status_code != 200:
            print("Error:", response.status_code, response.text)
            break

        data = response.json()
        results = data.get('results', [])
        all_results.extend(results)

        if len(results) < params['limit']:
            break

        params['page'] += 1

        if params['page'] > 1:  # Stop after fetching 3 pages
            break

    
    print(f"Total records retrieved: {len(all_results)}")
    print(f'extract_output {all_results[:3]}')
    return all_results 


'''url = 'https://api.openaq.org/v3/sensors/4217/days/monthly'
all_results = extract_data (url)
test_df = pd.DataFrame(all_results)
print(test_df.head(3))
test_df.to_csv('pollution_data/5_test_sensor.csv', index= True)'''

Total records retrieved: 110
extract_output [{'value': 18.1, 'flagInfo': {'hasFlags': False}, 'parameter': {'id': 5, 'name': 'no2', 'units': 'µg/m³', 'displayName': None}, 'period': {'label': '1 month', 'interval': '1 month', 'datetimeFrom': {'utc': '2015-12-31T23:00:00Z', 'local': '2016-01-01T00:00:00+01:00'}, 'datetimeTo': {'utc': '2016-01-31T23:00:00Z', 'local': '2016-02-01T00:00:00+01:00'}}, 'coordinates': None, 'summary': {'min': 16.386956521739126, 'q02': 16.454417391304343, 'q25': 17.230217391304347, 'median': 18.073478260869564, 'q75': 18.916739130434784, 'q98': 19.69253913043479, 'max': 19.760000000000005, 'avg': 18.073478260869564, 'sd': 2.385101916715329}, 'coverage': {'expectedCount': 31, 'expectedInterval': '744:00:00', 'observedCount': 2, 'observedInterval': '48:00:00', 'percentComplete': 6.0, 'percentCoverage': 6.0, 'datetimeFrom': {'utc': '2016-01-29T23:00:00Z', 'local': '2016-01-30T00:00:00+01:00'}, 'datetimeTo': {'utc': '2016-01-31T23:00:00Z', 'local': '2016-02-01T00:

In [ ]:
# Getting worldwide sensor locations
url= "https://api.openaq.org/v3/locations"
locations_raw = extract_data(url)
df_locations_raw = pd.DataFrame(locations_raw)
df_locations_raw.rename(columns={'id': 'location_id', 'name': 'location_name'}, inplace=True)
print(f'df_locations_raw output:\n {df_locations_raw.head(3)}')
df_locations_raw.to_csv('pollution_data/1_locations_raw.csv', index=True)

In [ ]:
#Unpack countries and coordinates
df_country = df_locations_raw['country'].apply(pd.Series)
df_coordinates = df_locations_raw['coordinates'].apply(pd.Series)
df_country.columns= ['country_id', 'country_code', 'country_name']
df_coordinates.columns= ['latitude', 'longitude']

#concatenate with location
df_locations_unpacked=pd.concat([df_locations_raw,df_country, df_coordinates],axis=1)
print(f'df_locations_unpacked output:\n {df_locations_unpacked[:3]}')
df_locations_unpacked.to_csv('pollution_data/2_locations_unpacked.csv')

In [ ]:
# keep only useful columns
df_locations_unpacked.drop(['owner', 'provider', 'isMobile', 'isMonitor', 'instruments', 'licenses', 'bounds', 'distance', 'country', 'coordinates'], axis=1, inplace=True,errors='ignore')
df_locations_clean = df_locations_unpacked
print(f'df_locations_clean output:\n{df_locations_clean.head(3)}')
df_locations_clean.to_csv('pollution_data/3_locations_raw_clean.csv', index= True)

In [ ]:
# unpack the sensor id
df_exploded = df_locations_clean.explode("sensors")
df_exploded["sensor_id"] = df_exploded["sensors"].apply(lambda x: x.get("id") if isinstance(x, dict) else None)
df_exploded.drop('sensors',axis=1, inplace=True)
print (df_exploded.head(3))
df_exploded.to_csv('pollution_data/3_locations_exploded.csv', index= True)

In [48]:
# generate URLs list
urls_list = [f"https://api.openaq.org/v3/sensors/{sensor_id}/days/monthly" for sensor_id in df_exploded['sensor_id']]
#urls_list = [f"https://api.openaq.org/v3/sensors/{sensor_id}/measurements" for sensor_id in df_exploded['sensor_id']]
print(len(urls_list))
print(f'url_list {urls_list[:3]}')

2719
url_list ['https://api.openaq.org/v3/sensors/6/days/monthly', 'https://api.openaq.org/v3/sensors/5/days/monthly', 'https://api.openaq.org/v3/sensors/7/days/monthly']


In [61]:
# generate sensor data  dataframe

data_results = []
sensor_ids = []
for url in urls_list[:50] : 
    sensor_id = int(url.split("/")[-3])
    sensors_data = extract_data(url)
    for record in sensors_data:
        record["sensor_id"] = sensor_id
        data_results.append(record)
    
    data_df = pd.DataFrame(data_results)

    if 'period' in data_df.columns:
    # Safely extract 'local' datetime
        data_df['year_month'] = data_df['period'].apply(
            lambda x: pd.to_datetime(x['datetimeFrom']['local']).strftime('%Y-%m') if isinstance(x, dict) and 'datetimeFrom' in x else None
        )

    # Display output
        print(f'data df output:{data_df[:3]}')
        print(f'data df output length:{len(data_df)}')

    # Save to CSV
        data_df.to_csv('pollution_data/4_sensor_data.csv', index=True)
    else:
        print("The 'period' column is missing in the DataFrame.")



Total records retrieved: 0
extract_output []
The 'period' column is missing in the DataFrame.
Total records retrieved: 0
extract_output []
The 'period' column is missing in the DataFrame.
Total records retrieved: 0
extract_output []
The 'period' column is missing in the DataFrame.
Total records retrieved: 0
extract_output []
The 'period' column is missing in the DataFrame.
Total records retrieved: 0
extract_output []
The 'period' column is missing in the DataFrame.
Total records retrieved: 0
extract_output []
The 'period' column is missing in the DataFrame.
Total records retrieved: 0
extract_output []
The 'period' column is missing in the DataFrame.
Total records retrieved: 0
extract_output []
The 'period' column is missing in the DataFrame.
Total records retrieved: 0
extract_output []
The 'period' column is missing in the DataFrame.
Total records retrieved: 0
extract_output []
The 'period' column is missing in the DataFrame.
Total records retrieved: 0
extract_output []
The 'period' co

In [ ]:
'''# get the sensor ID list  along with the location id

location_sensor_pairs = [
    (location["id"], sensor["id"], location["coordinates"]["latitude"], location["coordinates"]["longitude"])
    for location in locations_data
    for sensor in location["sensors"]
]

# Convert to DataFrame
df_locations_sensors_pairs = pd.DataFrame(
    location_sensor_pairs,
    columns=["location_id", "sensor_id", "location_latitude", "location_longitude"]
)

# Display first 3 rows
print(f'locations_sensors_pairs_list output : {location_sensor_pairs[:4]}')
print(f'locations_sensors_pairs_df output :\n{df_locations_sensors_pairs.head(4)}')

# Save to CSV
df_locations_sensors_pairs.to_csv("pollution_data/locations_sensore_list.csv", index=False)


print(f'locations_sensors_pairs_list output :  {location_sensor_pairs[:3]}')
df_locations_sensors_pairs = pd.DataFrame(location_sensor_pairs, columns=["location_id", "sensor_id", 'location_latitude', 'location_longitude'])
print(f'locations_sensors_pairs_df output :  {df_locations_sensors_pairs[:3]}')
df_locations_sensors_pairs.to_csv("pollution_data/locations_sensore_list.csv", index=False)

  # generate the location id list and dataframe 
sensor_id_list= [location[0] for location in  location_sensor_pairs]
print(sensor_id_list[:3])
sensor_id_df = pd.DataFrame(sensor_id_list)
print(f'sensor_id_df {sensor_id_df.head(3)}')

#conacetenate sensor data with location ID
final_df = pd.concat([sensor_id_df, data_df], axis=1)
print (f'final df:{final_df[:3]}')
print (f'final df legnth:{len(final_df)}')'''





In [ ]:

#url = "https://api.openaq.org/v3/sensors/3917/days/monthly"
#url_1 = "https://api.openaq.org/v3/sensors/3917"
#url_2 = "https://api.openaq.org/v3/sensors/3917/measurements"
#url = "https://api.openaq.org/v3/locations?bbox=-10.5,36,9.6,51.1"
#url = "https://api.openaq.org/v3/sensors/3917/days/monthly"
#url = "https://api.openaq.org/v3/sensors/3917/years"
#url = "https://api.openaq.org/v3/sensors/3917/days"
#url = "https://api.openaq.org/v3/sensors/3917/measurements"
#url = "https://api.openaq.org/v3/parameters"
#url = "https://api.openaq.org/v3/locations"
#url = "https://api.openaq.org/v3/countries"



